# Collaborative Filtering Recommender System - Expedia Hotel dataset

## Import Libraries

In [39]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import train_test_split
import warnings
import implicit
from scipy.sparse import coo_matrix
import scipy.sparse as sparse

import sys
#sys.path.append('/Users/yas/Downloads/github/recommender_system')

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from math import sqrt

from scipy.sparse import csr_matrix
from scipy.sparse import coo_matrix
import implicit

In [40]:
hotel_train_set = pd.read_csv('/Users/yas/Downloads/github/recommender_system/data/hotel_data/train.csv', sep=',', nrows=100000)
hotel_train_set.shape

(100000, 24)

### Read train and test data

In [41]:
hotel_train_set.head(n=2)

,date_time,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,is_mobile,is_package,...,srch_children_cnt,srch_rm_cnt,srch_destination_id,srch_destination_type_id,is_booking,cnt,hotel_continent,hotel_country,hotel_market,hotel_cluster
0,2014-08-11 07:46:59,2,3,66,348,48862,2234.2641,12,0,1,...,0,1,8250,1,0,3,2,50,628,1
1,2014-08-11 08:22:12,2,3,66,348,48862,2234.2641,12,0,1,...,0,1,8250,1,1,1,2,50,628,1


In [42]:
len(hotel_train_set['srch_destination_type_id'].unique()),len(hotel_train_set['hotel_cluster'].unique())

(8, 100)

In [43]:
#df = hotel_train_set[['user_id','srch_destination_id','is_booking']]
df = hotel_train_set[['user_id','hotel_cluster','is_booking']]

In [44]:
df.shape

(100000, 3)

In [45]:
#rename columns
df.columns =['user_id', 'item_id', 'rating']

In [46]:
# for user 12 and item 12 we have 3 values
df.head()

,user_id,item_id,rating
0,12,1,0
1,12,1,1
2,12,1,0
3,93,80,0
4,93,21,0


### Remove rows with the same user_id and item_id and different rating

In [47]:
max_rating = df.groupby(['user_id', 'item_id']).rating.transform(max)
df = df.loc[df.rating == max_rating]
df.drop_duplicates(keep='first',inplace=True) 

In [48]:
df.shape

(40811, 3)

In [49]:
train, test = train_test_split(df, test_size=0.3)

train.shape, test.shape

((28567, 3), (12244, 3))

# Find Similar Hotel clusters

In [50]:
ratings = pd.DataFrame(df.groupby('item_id')['rating'].mean())
ratings.head()

,rating
item_id,
0,0.117794
1,0.252396
2,0.209877
3,0.113043
4,0.184035


In [51]:
ratings['number_ratings'] = pd.DataFrame(df.groupby('item_id')['rating'].count())
ratings.head()

,rating,number_ratings
item_id,,
0,0.117794,399
1,0.252396,313
2,0.209877,486
3,0.113043,345
4,0.184035,451


In [52]:
hotel_matrix = df.pivot_table(index='user_id',columns='item_id',values='rating')

In [53]:
hotel_matrix.head()

item_id,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
user_id,,,,,,,,,,,,,,,,,,,,,
12,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
93,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
501,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
756,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
776,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [54]:
def find_similar_clusters(cluster_number):
    #Select user ratings for twohotel_matrixmovies 
    item_user_ratings = hotel_matrix[cluster_number]

    # Find correlations between series with corrwith (instead of corr)
    similar_to_hotel = hotel_matrix.corrwith(item_user_ratings)

    # Removing NaN values and using a DataFrame instead of a series 
    corr_hotel = pd.DataFrame(similar_to_hotel,columns=['Correlation'])
    corr_hotel.dropna(inplace=True)

    corr_hotel = corr_hotel.join(ratings['number_ratings'])

    result = corr_hotel[corr_hotel['number_ratings']>0].sort_values('Correlation',ascending=False).head()
    return result

In [55]:
warnings.filterwarnings("ignore")
find_similar_clusters(11)

,Correlation,number_ratings
item_id,,
11,1.000000,411
66,0.469042,195
57,0.459353,304
35,0.404846,191
32,0.401742,336


# Recommendation Engine - collaborative filtering model from scratch

## Memory-Based CF by computing cosine similarity

In [56]:
n_users = df.user_id.unique().shape[0]
n_items = df.item_id.unique().shape[0]
n_users,n_items

(3478, 100)

In [57]:
data_matrix = np.zeros((n_users, n_items))
data_matrix = df.pivot(index='user_id', columns='item_id', values='rating').fillna(0)
data_matrix=data_matrix.as_matrix()

train_data_matrix = np.zeros((n_users, n_items))
train_data_matrix = train.pivot(index='user_id', columns='item_id', values='rating').fillna(0)
train_data_matrix=train_data_matrix.as_matrix()

test_data_matrix = np.zeros((n_users, n_items))
test_data_matrix = test.pivot(index='user_id', columns='item_id', values='rating').fillna(0)
test_data_matrix=test_data_matrix.as_matrix()

In [58]:
train_data_matrix.shape, test_data_matrix.shape

((3421, 100), (2944, 100))

In [59]:
# calculating the similarity by using the pairwise_distance from sklearn to calculate the cosine similarity
from sklearn.metrics.pairwise import pairwise_distances 

# user-user similarity
user_similarity = pairwise_distances(train_data_matrix, metric='cosine')

# item-item similarity
item_similarity = pairwise_distances(train_data_matrix.T, metric='cosine')

user_similarity.shape , item_similarity.shape

((3421, 3421), (100, 100))

we can make a prediction by applying the following formula for user-based CF.


1) We can look at the similarity between users k and a as weights

2)  weights are multiplied by the ratings of a similar user a (corrected for the average rating of that user)

3) We need to normalize it so that the ratings stay between 1 and 5

4) As a final step, sum the average ratings for the user that you are trying to predict.

$$\hat{x}_{k,m}= \bar{x}_{k} + \frac{\sum_{u_{a}}sim_{u}(u_{k},u_{a})(x_{a,m}-\bar{x}_{u_{a}})}
{\sum_{u_{a}\left | sim_{u}(u_{k},u_{a}) \right |}}
$$

Also, we can make a prediction by applying the following formula for item-based CF.

 $$\hat{x}_{k,m}=  \frac{\sum_{i_{b}}sim_{i}(i_{m},i_{b})(x_{k,b})}
{\sum_{i_{b}\left | sim_{i}(i_{m},i_{b}) \right |}}
$$

In [60]:
#make predictions based on these similarities

def predict(ratings, similarity, type='user'):
    if type == 'user':
        mean_user_rating = ratings.mean(axis=1)
        #We use np.newaxis so that mean_user_rating has same format as ratings
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis])
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])
    return pred

In [61]:
user_prediction = predict(train_data_matrix, user_similarity, type='user')
item_prediction = predict(train_data_matrix, item_similarity, type='item')

user_prediction.shape,item_prediction.shape,test_data_matrix.shape

((3421, 100), (3421, 100), (2944, 100))

## Model-Based CF by using singular value decomposition (SVD)

$$X = USV^{T}$$

sparsity = round(1.0-len(df)/float(n_users*n_items), 3)
print('The sparsity level of MovieLens100K is ' + str(sparsity*100) + '%')


import scipy.sparse as sp
from scipy.sparse.linalg import svds

#get SVD components from train matrix. Choose k.
u, s, vt = svds(train_data_matrix, k=20)
s_diag_matrix=np.diag(s)

#prediction
X_pred = np.dot(np.dot(u, s_diag_matrix), vt)

## Model-Based CF by using ALS
Alternating Least Squares is a form of matrix factorization that reduces this user-item matrix to a much smaller amount of dimension called latent or hidden features

In [37]:
df

,user_id,item_id,rating
1,12,1,1
3,93,80,0
4,93,21,0
5,93,92,0
6,501,41,0
...,...,...,...
99943,391007,14,0
99949,391007,93,0
99955,391007,57,0
99956,391007,20,0


In [39]:
# Create lists of all users, artists and plays
users = list(np.sort(df.user_id.unique()))
artists = list(np.sort(df.item_id.unique()))
plays = list(df.rating)

# Get the rows and columns for our new matrix
rows = df.user_id.astype(int)
cols = df.item_id.astype(int)

#data_sparse = sparse.csr_matrix((plays, (rows, cols)), shape=(len(users), len(artists)))


In [44]:
train_data_matrix2 = train_data_matrix.nonzero()
train_data_matrix2

(array([   0,    3,    5, ..., 3410, 3411, 3413]),
 array([ 1,  2, 36, ..., 91, 15, 93]))

We start out by calculating the confidence for all users and items, create our X and Y matrices to hold our user and item vectors and randomly assign the values. We also precompute our I diagonals.

In [46]:
def implicit_als(sparse_data, alpha_val=40, iterations=10, lambda_val=0.1, features=10):
 
    '''We teratively
    compute the user (x_u) and item (y_i) vectors using the following formulas:
 
    x_u = ((Y.T*Y + Y.T*(Cu - I) * Y) + lambda*I)^-1 * (X.T * Cu * p(u))
    y_i = ((X.T*X + X.T*(Ci - I) * X) + lambda*I)^-1 * (Y.T * Ci * p(i))
 '''
    '''Args:
        sparse_data (csr_matrix): Our sparse user-by-item matrix
 
        alpha_val (int): The rate in which we'll increase our confidence
        in a preference with more interactions.
 
        iterations (int): How many times we alternate between fixing and 
        updating our user and item vectors
 
        lambda_val (float): Regularization value
 
        features (int): How many latent features we want to compute.
    
    Returns:     
        X (csr_matrix): user vectors of size users-by-features
        
        Y (csr_matrix): item vectors of size items-by-features'''
   

    confidence = train_data_matrix * alpha_val

    # Get the size of user rows and item columns
    user_size, item_size = sparse_data.shape

    # We create the user vectors X of size users-by-features, the item vectors
    # Y of size items-by-features and randomly assign the values.
    X = sparse.csr_matrix(np.random.normal(size = (user_size, features)))
    Y = sparse.csr_matrix(np.random.normal(size = (item_size, features)))

    #Precompute I and lambda * I
    X_I = sparse.eye(user_size)
    Y_I = sparse.eye(item_size)

    I = sparse.eye(features)
    lI = lambda_val * I

## Evaluation with RMSE and MAE

In [247]:
user_prediction

array([[-0.00300322,  0.00051275,  0.00754468, ...,  0.00314972,
         0.00813068,  0.00139174],
       [-0.00300322,  0.00051275,  0.00754468, ...,  0.00314972,
         0.00813068,  0.00139174],
       [ 0.00712222,  0.01078529,  0.00596284, ...,  0.01248412,
         0.01794034,  0.01067933],
       ...,
       [ 0.00708326,  0.01079847,  0.01755057, ...,  0.013416  ,
         0.01842785,  0.01149777],
       [ 0.00713507,  0.0106716 ,  0.01740528, ...,  0.01338125,
         0.01836329,  0.01129372],
       [-0.00300322,  0.00051275,  0.00754468, ...,  0.00314972,
         0.00813068,  0.00139174]])

In [240]:
#test_data_matrix = test_data_matrix.as_matrix()

In [241]:
#np.nonzero(test_data_matrix)

In [242]:
#test_data_matrix.nonzero()

In [243]:
test_data_matrix.shape,user_prediction.shape

((2965, 100), (3414, 100))

In [248]:
def rmse(prediction, y):
    prediction = prediction[y.nonzero()].flatten()
    y = y[y.nonzero()].flatten()
    return sqrt(mean_squared_error(prediction, y))

def mae(prediction, y):
    prediction = prediction[y.nonzero()].flatten()
    y = y[y.nonzero()].flatten()
    return sqrt(mean_absolute_error(prediction, y))

print('User-based CF RMSE: ' + str(rmse(user_prediction, test_data_matrix)))
print('Item-based CF RMSE: ' + str(rmse(item_prediction, test_data_matrix)))
print('Model-Based CF RMSE: ' + str(rmse(X_pred, test_data_matrix)))
print("\n")
print('User-based CF MAE: ' + str(mae(user_prediction, test_data_matrix)))
print('Item-based CF MAE: ' + str(mae(item_prediction, test_data_matrix)))
print('Model-Based CF MAE: ' + str(mae(X_pred, test_data_matrix)))

User-based CF RMSE: 0.9822616152482693
Item-based CF RMSE: 0.9868935766520119
Model-Based CF RMSE: 0.9868423753854999


User-based CF MAE: 0.9909410940593059
Item-based CF MAE: 0.9932974140318174
Model-Based CF MAE: 0.9907364650766246


## Evaluation with Precision and recall
Precision and recall are binary metrics used to evaluate models with binary output. 

We need a way to translate the ratings from 1 to 5 into a binary problem.

To do the translation we will assume that any true rating above 3.5 corresponds to a relevant item and any true rating below 3.5 is irrelevant. 

We are intrested in recommending top-N items to the user. So it makes more sense to compute precision and recall metrics in the first N items instead of all the items.

Thus the notion of precision and recall at k where k is a user definable integer that is set by the user to match the top-N recommendations objective.

In [ ]:
# threshold = 3.5


### Recommend hotel cluster for a user

In [174]:
rows = []
movies = list(df.drop_duplicates(subset='item_id', keep='first').item_id)
names = list(df.drop_duplicates(subset='item_id', keep='first').item_id)

for i in range(len(movies)):
    rows.append( dict({'name': names[i], 'item_id' : movies[i], 'user_id' : 9}))
test_data = pd.DataFrame(rows)

In [175]:
for idx, (score, name) in enumerate(sorted(zip(preds, names), reverse=True)):
    if idx >= 10:
        break
    print("Score: ", round(float(score), 2), " for hotel cluster: ", name)

Score:  0.41  for hotel cluster:  33
Score:  0.31  for hotel cluster:  45
Score:  0.31  for hotel cluster:  34
Score:  0.3  for hotel cluster:  81
Score:  0.29  for hotel cluster:  32
Score:  0.25  for hotel cluster:  96
Score:  0.25  for hotel cluster:  28
Score:  0.23  for hotel cluster:  15
Score:  0.23  for hotel cluster:  88
Score:  0.22  for hotel cluster:  29


# Resources

https://blog.cambridgespark.com/nowadays-recommender-systems-are-used-to-personalize-your-experience-on-the-web-telling-you-what-120f39b89c3c

https://course.fast.ai/videos/?lesson=4

https://towardsdatascience.com/collaborative-filtering-with-fastai-3dbdd4ef4f00

https://medium.com/quantyca/deep-learning-for-collaborative-filtering-using-fastai-b28e197ccd59